In [ ]:
# standars imports
import numpy as np
import pandas as pd
# visuals
import matplotlib.pyplot as plt
import seaborn as sns
# date/time
from datetime import timedelta, datetime
from sklearn.model_selection import TimeSeriesSplit
# stats
import statsmodels.api as sm
# .py files
import prepare
import acquire
# plt.style.use('seaborn-whitegrid')
plt.rc('figure', figsize=(13, 7))
plt.rc('font', size=16)
# notebook formatting
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# read in and prep the store data:
df = acquire.wrangle_store_data()
df = prepare.prep_store_data(df)

In [ ]:
df.head()

In [ ]:
# max and min dates
df.index.min(), df.index.max()

In [ ]:
# train, validate, test split:

train = df[:'2015']
validate = df['2016']
test = df['2017']

In [ ]:
# visualize the split data:

train.resample('D').sales_total.mean().plot()
validate.resample('D').sales_total.mean().plot()
test.resample('D').sales_total.mean().plot()

In [ ]:
# y is the resampled daily data: 

y = train.resample('D').sales_total.mean()
y.head()

In [ ]:
#plot the daily differences
# observe weather usage seems to vary drastically from day to day or has more of a smooth transition

y.diff()['2013'].plot()
plt.title('Daily Sales Difference')
plt.ylabel('Difference in sales from one day to the next')

In [ ]:
# closer look at those spikes
y.diff()['2013-02-01':'2013-05-01'].plot()
plt.title('Daily Sales Differences')
plt.ylabel('Differences in sales from one day to the next')

In [ ]:
# maybe it would be helpful to look at the percentage change as well

(y.diff() / y.shift()).plot(alpha=.5)
plt.title('Daily % Change in Sales Total')
plt.ylabel('% Change')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True)

weekly = y.resample('W').mean()
(weekly.diff() / weekly.shift()).plot(ax=ax1)
ax1.set(title='Weekly % Change in Sales Total', ylabel='% Change', xlabel='')

monthly = y.resample('M'),mean()
(monthly.diff() / monthly.shift()).plot(ax=ax2)
ax2.set(title='Monthly % Change in Sales Total')
# ax2.hlines(0, *ax2.get_xlim())

In [ ]:
y.resample('W').mean()

In [ ]:
# autocorrelation after weekly resampling:
 pd.plotting.autocorrelation_plot(y.resample('W').mean())

In [ ]:
# decompose the data
sm.tsa.seasonal_decompose(y.resample('W').mean()).plot()

In [ ]:
power = pd.read_csv("https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv")
power = prepare.prep_opsd_data(power)

In [ ]:
power.head()

In [ ]:
# number of nulls
power.isnull().sum()

In [ ]:
power.index.min(), power.index.max()

In [ ]:
# split data
train = power[:'2013']
train.head()

In [ ]:
# plot the consumption, wind and solar data

train.consumption.plot(label="consumption")
plt.plot(train['consumption'].rolling('90D').mean(), label="Rolling Mean")
train.wind.plot()
train.solar.plot()
plt.legend()

In [ ]:
# lets look at consumption in a single year and see if we see any pattern
plt.figure(figsize=(12,8))
plt.plot(train.loc['2010-01':'2010-12']['consumption'], linewidth = 0.5)
plt.title("consumption in 2010")
plt.show()

In [ ]:
# resample monthly and plot
train.consumption.resample('M').mean().plot()

In [ ]:
# resample data on yearly basis and plot
train.consumption.resample('Y').mean().plot()
plt.ylim(1100,1400)

In [ ]:
# lets fill all NaNs with zero
train = train.fillna(0)

In [ ]:

# make a new datafrsame sources
sources = train[['consumption', 'wind', 'solar']]

# make a new column called conventional: Consumption - Wind - Solar
sources['conventional'] = sources.consumption - sources.wind - sources.solar

In [ ]:

# lets plot the three power sources as stacked bar chart:

ax = sources[['conventional', 'wind', 'solar']].resample('Y').sum().plot.bar(stacked = True)
ax.set_ylim(0, 700000)
ax.legend(ncol = 3)

labels = [pd.to_datetime(t.get_text()).strftime('%Y') for t in ax.get_xticklabels()]
ax.set_xticklabels(labels);

In [ ]:
train['month'] = train.index.month
train['weekday'] = train.index.day_name()

In [ ]:
# energy consumption by month
sns.boxplot(data= train, x='month', y='consumption')

In [ ]:
# energy consumption by weekday
sns.boxplot(data= train, x='weekday', y='consumption')

In [ ]:
# subseasonal plot
y = train.consumption
table = y.groupby([y.index.strftime('%Y'), y.index.strftime('%m-%b')]).mean().unstack()

In [ ]:
fig, axs = plt.subplots(1, 12, sharey=True, sharex=True, figsize=(25,8))
for ax, (month, subset) in zip(axs, table.iteritems()):
    subset.plot(ax=ax. title=month)
    ax.hlines(subset.mean(), x_left, x_right, ls='--')
    ax.set(xlabel='')
    ax.tick_params('x', rotation=45)

fig.suptitle('Seasonal Subseries Plot') # super-title for the overall figure
fig.subplots_adjust(wspace=0)

In [ ]:
fig, axs = plt.subplots(3, 4, sharex=False, sharey=True, figsize=(25,8))

for ax, (month, subset) in zip(axs.ravel(), table.iteritems()):
    subset.plot(ax=ax, title=month[3:])
    ax.hlines(subset.mean(), *ax.get_xlim(), ls = '--')
fig.subplots_adjust(wspace=0)
plt.tight_layout()

In [ ]:
# How does the wind production/consumption vary within a year and also year to year

ax = train['2010':].wind.resample('M').sum().plot()
ax.set(title = 'Wind production pattern')

In [ ]:
# bring in the gdp growth rate data

growth_rate = {2006:3.815, 2007:2.984, 2008:0.962, 2009:-5.697, 2010:4.179, 2011:3.924, 2012:0.42, 2013:0.428, 2014:2.226 , 2015:1.74 , 2016:2.23 , 2017:2.465 }

In [ ]:
# convert dict to dataframe
growth_rate = pd.DataFrame.from_dict(growth_rate, orient='index', columns = ['growth_rate'])
growth_rate.head()

In [ ]:
# set index as datetimeindex
growth_rate.index= pd.to_datetime(growth_rate.index, format = '%Y')

In [ ]:
# check the dataframe head
growth_rate.head()

In [ ]:
# compare power consumption and GDP growth rate:

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16, 9))

ax = power.consumption.resample('Y').sum().plot(figsize = (12,6), ax = ax1)
ax1.set(title='Power Consumption')
ax1.set_ylabel('Power Consumption')

growth_rate.plot(ax =ax2)
ax2.set(title='GDP Growth Rate')
ax2.set_ylabel('Growth Rate')

In [ ]:
decomposition = sm.tsa.seasonal_decompose(train.resample('W').consumption.mean(), model='additive')

In [ ]:
decomposition.plot()
plt.show()

In [ ]:
# autocorrelation plot
pd.plotting.autocorrelation_plot(train.consumption.resample('W').mean())

In [ ]:
# is there a weekly seasonality in this data
pd.plotting.autocorrelation_plot(train.consumption['2010-01':'2010-03'])